# Import


In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
from sklearn.preprocessing import MinMaxScaler,StandardScaler 
from sklearn.model_selection import train_test_split
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# Read Data

In [2]:
develop = pd.read_csv("CORBL_2000-01-01_2021-12-31.csv", engine='python')
develop.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
0,1,2021-12-29,71,4798.0,3958278.7,851.4,815.0,851.4
1,2,2021-12-28,56,2596.0,2163466.2,856.0,821.3,849.9
2,3,2021-12-27,82,8257.0,6898740.4,877.0,825.0,856.0
3,4,2021-12-26,58,3820.0,3258775.5,875.0,842.8,860.0
4,5,2021-12-23,103,9501.0,8142395.8,890.3,849.7,851.0


# Sort Data In Ascending Order

In [3]:
develop = develop.sort_values(by="Date")
develop.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
861,862,2011-08-17,1,10.0,3820.0,382.0,382.0,382.0
860,861,2011-11-09,1,10.0,3750.0,375.0,375.0,375.0
859,860,2011-11-13,3,70.0,25130.0,368.0,354.0,354.0
858,859,2011-11-14,2,40.0,13700.0,347.0,341.0,341.0
857,858,2011-11-15,2,40.0,13220.0,335.0,329.0,329.0


# Handle Missing NAs

In [4]:
develop = develop.dropna()
develop.tail()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
4,5,2021-12-23,103,9501.0,8142395.8,890.3,849.7,851.0
3,4,2021-12-26,58,3820.0,3258775.5,875.0,842.8,860.0
2,3,2021-12-27,82,8257.0,6898740.4,877.0,825.0,856.0
1,2,2021-12-28,56,2596.0,2163466.2,856.0,821.3,849.9
0,1,2021-12-29,71,4798.0,3958278.7,851.4,815.0,851.4


# Plot Trend

In [5]:
develop.iplot(kind="line",x="Date",y="Close Price",xTitle="Date", yTitle="Close Price", title="Development Bank Trend")

# Next Price

In [6]:
develop['Next Price'] = develop.shift(-1)['Close Price']

In [7]:
develop = develop.dropna()

# Rename Columns

In [8]:
develop.columns

Index(['S.N.', 'Date', 'Total Transactions', 'Total Traded Shares',
       'Total Traded Amount', 'Max. Price', 'Min. Price', 'Close Price',
       'Next Price'],
      dtype='object')

In [9]:
develop.rename(columns = {'S.N.':'SN','Total Transactions':'TTrans', 'Total Traded Shares':'TTS',
                              'Total Traded Amount':'TTA','Close Price':'ClosePrice','Next Price':'NextPrice',
                     'Max. Price':'MaxPrice','Min. Price':'MinPrice'}, inplace = True)

# Prepared Feature


In [10]:
features = ['ClosePrice',"Date"]
outFeature = ['NextPrice',"Date"]

In [11]:
# take close price as feature and next price as out feature
X, Y = develop[features], develop[outFeature] 
X.set_index("Date", inplace=True)
Y.set_index("Date", inplace=True)

# Normalized Data

In [12]:
#Normalize data using standard scalar.
ss = StandardScaler()

X["ClosePrice"] = ss.fit_transform(X)
Y["NextPrice"] = ss.fit_transform(Y)


# Train Test Split

Using train_test_split of sklearn will split data randomly so the sequence is lost.

In [13]:
def train_test_split(x,y,train_size):
    n = int(len(x)*train_size)
    trainx = x[:n]
    testx = x[n:]
    trainy=y[:n]
    testy=y[n:]
    return trainx,testx,trainy,testy

In [14]:
#Split train, validation, test set from X,Y to 80%, 10%, 10%

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.9)

In [15]:
X_train.shape,X_test.shape,X.shape

((774, 1), (87, 1), (861, 1))

# Convert to Matrix


In [16]:
# convert into dataset matrix
def convertToMatrix(data, step):
    data = data.to_numpy()
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

step=5

trainX,trainY =convertToMatrix(X_train,step)
testX,testY =convertToMatrix(X_test,step)
# validX,validY = convertToMatrix(X_valid,step)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1],1))
# validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1],1))

trainX.shape,testX.shape #,validX.shape

((769, 5, 1), (82, 5, 1))

# Modeling

In [17]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(5,1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

In [18]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(trainX, trainY, batch_size=1, epochs=1)

769/769 [==============================] - 12s 6ms/step - loss: 0.0218


In [19]:
predictions = model.predict(testX)
predictions = ss.inverse_transform(predictions)
predictions.shape,testX.shape

3/3 [==============================] - 2s 6ms/step


((82, 1), (82, 5, 1))

# RMSE Of Test

In [20]:
Y_test_ss = ss.inverse_transform(Y_test)[:-5]

rmse=np.sqrt(np.mean(((predictions- Y_test_ss)**2)))
print(rmse)

98.30160811743332


# Test Trend

In [21]:
test=pd.DataFrame()
test["pred"]=ss.inverse_transform(predictions.flatten())
test["real"]=ss.inverse_transform(Y_test_ss)
test["date"]=X_test.index[:-5]
test.iplot(kind="line", x="date")